# Mejora gráficos por semana epidemiológica

Prueba con muestra `categoricas` y `númericas`

In [2]:
# Importa librerias necesarias para el análisis

import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
# Importa datos de población regional y crea DF


df_poblacion = pd.read_csv('data/aysen_population.csv') #Archivo ubicado dentro de la carpeta data

df_poblacion.head()

,Unnamed: 0,orden,nombre_region,codigo_region,nombre_provincia,codigo_provincia,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,0,7172,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Coyhaique,11101,28763,29055,57818
1,1,7194,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Lago Verde,11102,494,358,852
2,2,7216,Región de Aysén del General Carlos Ibáñez del ...,11,Aysen,112,Aysen,11201,12719,11240,23959
3,3,7238,Región de Aysén del General Carlos Ibáñez del ...,11,Aysen,112,Cisnes,11202,3922,2595,6517
4,4,7260,Región de Aysén del General Carlos Ibáñez del ...,11,Aysen,112,Guaitecas,11203,1166,677,1843


In [4]:
# Importa datos de comunas corregidas por establecimiento de salud

df_establecimiento = pd.read_excel('data/establecimientos_salud_dict.xlsx') #Archivo ubicado dentro de la carpeta data


df_establecimiento.head()

,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi
0,Actividades gestionadas por la Dirección del S...,Coyhaique,Coyhaique
1,Centro Clínico Militar Coyhaique,Coyhaique,Coyhaique
2,Centro Comunitario de Salud Familiar Villa Ma...,Aysen,Aysen
3,Centro Comunitario de Salud Familiar Alejandro...,Coyhaique,Coyhaique
4,Centro Comunitario de Salud Familiar Puerto Ch...,Aysen,Aysen


In [5]:
# Importa datos desde la base de datos recivida via FTP
## PRIMERO: Guardar archivo dentro de la carpeta data
## SEGUNDO: No cambiar nombre ni formato del archivo recibido
## TERCERO: RECORDAR CAMBIAR FECHA EN EL ARCHIVO A IMPORTAR A CONTINUACION, i.e: 20210829 para este domingo 29 de Agosto 2021

df = pd.read_csv('data/20210826_Región de Aysén del General Carlos Ibáñez del Campo_notificaciones_0600.csv',# RECORDAR CAMBIAR FECHA EN EL ARCHIVO 
                   sep=";", # CSV con separador de semicolon
                   low_memory=False
                  )

In [6]:
dff = df.merge(df_establecimiento)

In [7]:
df_merged = dff.merge(df_poblacion.drop(columns= ['Unnamed: 0','orden','nombre_region', 'codigo_region', 'nombre_provincia','codigo_provincia']), left_on='comuna_seremi_xi', right_on='nombre_comuna' )

In [8]:
# Filtra datos obtenidos solo para pacientes con residencia en la Región de Aysén

data_aysen = df_merged[df_merged['region_residencia'] == 'Región de Aysén del General Carlos Ibáñez del Campo']

data_aysen.head(1)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,ningun_factor_riesgo,test_antigeno_sars_positivo,tac_torax_covid,comuna_seremi_xi,provincia_seremi_xi,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,100166,50166,50,Coronavirus (COVID-19),11,2020-03-12,DESCARTADA,Hospital Regional (Coihaique),Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,NaN,NaN,NaN,Coyhaique,Coyhaique,Coyhaique,11101,28763,29055,57818


In [9]:
data_aysen = data_aysen[((data_aysen['estado_caso']=='Validada'
                        )|(data_aysen['estado_caso']=='Inconcluso'
                          ))&(data_aysen['vigente_no_eliminado']=='t')
                       ].rename(columns={'edad_hombres':'pop_hombres',
                                         'edad_mujeres':'pop_mujeres',
                                         'total':'pop_total'})

data_aysen.sample(3)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,ningun_factor_riesgo,test_antigeno_sars_positivo,tac_torax_covid,comuna_seremi_xi,provincia_seremi_xi,nombre_comuna,codigo_comuna,pop_hombres,pop_mujeres,pop_total
19563,14653252,14603252,50,Coronavirus (COVID-19),27,2021-07-05,DESCARTADA,Hospital Regional (Coihaique),Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,t,DESCONOCIDO,DESCONOCIDO,Coyhaique,Coyhaique,Coyhaique,11101,28763,29055,57818
14457,10529958,10479958,50,Coronavirus (COVID-19),14,2021-04-06,DESCARTADA,Hospital Regional (Coihaique),Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,t,DESCONOCIDO,DESCONOCIDO,Coyhaique,Coyhaique,Coyhaique,11101,28763,29055,57818
56718,8281920,8231920,50,Coronavirus (COVID-19),7,2021-02-15,DESCARTADA,Actividades gestionadas por la Dirección del S...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,f,NO,NO,Coyhaique,Coyhaique,Coyhaique,11101,28763,29055,57818


In [10]:
# Selecciona variables de importancia

data_aysen_clean =  data_aysen[[
                                'numero_folio',
                                'comuna_residencia',
                                'fecha_notificacion',
                                'semana_epidemiologica',
                                'etapa_clinica',
                                'resultado_pcr_1',
                                'resultado_pcr_2',
                                'resultado_pcr_3',
                                'fecha_toma_muestra_1',
                                'fecha_toma_muestra_2',
                                'fecha_toma_muestra_3',
                                'fecha_resultado_pcr_1',
                                'fecha_resultado_pcr_2',
                                'fecha_resultado_pcr_3',
                                'establecimiento_salud',
                                'comuna_seremi_xi',
                                'provincia_seremi_xi',
                                'pop_hombres',
                                'pop_mujeres',
                                'pop_total'
                               ]]

In [11]:
# Transforma Fechas a formato DateTime 

data_aysen_clean['fecha_notificacion'] = pd.to_datetime(data_aysen_clean['fecha_notificacion']) 

#data_aysen_clean['fecha_toma_muestra_1'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_1'])

#data_aysen_clean['fecha_toma_muestra_2'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_2'])

data_aysen_clean['fecha_toma_muestra_3'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_3'])

data_aysen_clean['fecha_resultado_pcr_1'] = pd.to_datetime(data_aysen_clean['fecha_resultado_pcr_1'])

data_aysen_clean['fecha_resultado_pcr_2'] = pd.to_datetime(data_aysen_clean['fecha_resultado_pcr_2'])

data_aysen_clean['fecha_resultado_pcr_3'] = pd.to_datetime(data_aysen_clean['fecha_resultado_pcr_3'])

<ipython-input-11-ba7968d6c671>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean['fecha_notificacion'] = pd.to_datetime(data_aysen_clean['fecha_notificacion'])
<ipython-input-11-ba7968d6c671>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean['fecha_toma_muestra_3'] = pd.to_datetime(data_aysen_clean['fecha_toma_muestra_3'])
<ipython-input-11-ba7968d6c671>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [12]:
semana_x = 33

year = 2021

# Filtra datos para semana_x y año (year)

data_semx = data_aysen_clean[(data_aysen_clean['fecha_notificacion'].dt.year == year)&(data_aysen_clean['semana_epidemiologica']== semana_x)]

data_semx.sample(10)

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,fecha_toma_muestra_1,fecha_toma_muestra_2,fecha_toma_muestra_3,fecha_resultado_pcr_1,fecha_resultado_pcr_2,fecha_resultado_pcr_3,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi,pop_hombres,pop_mujeres,pop_total
129154,16239800,Chile Chico,2021-08-20,33,DESCARTADA,NEGATIVO,NEGATIVO,NaN,2021-07-21,2021-08-18,NaT,2021-07-21,2021-08-21,NaT,Hospital Dr. Leopoldo Ortega R. (Chile Chico),Chile Chico,General Carrera,2577,2288,4865
21737,16103239,Coyhaique,2021-08-17,33,DESCARTADA,NaN,NaN,NaN,2021-08-17,NaN,NaT,NaT,NaT,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,28763,29055,57818
109672,16182758,Aisén,2021-08-19,33,DESCARTADA,NEGATIVO,NaN,NaN,2021-08-19,NaN,NaT,2021-08-20,NaT,NaT,Centro de Salud Familiar Puerto Aysen,Aysen,Aysen,12719,11240,23959
21942,16199507,Coyhaique,2021-08-20,33,DESCARTADA,NEGATIVO,NaN,NaN,2021-08-20,2021-08-20,NaT,2021-08-20,NaT,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,28763,29055,57818
76441,16186463,Coyhaique,2021-08-19,33,DESCARTADA,NEGATIVO,NEGATIVO,NEGATIVO,2021-07-09,2021-08-07,2021-08-19,2021-07-10,2021-08-08,2021-08-20,Actividades gestionadas por la Dirección del S...,Coyhaique,Coyhaique,28763,29055,57818
109509,16108596,Aisén,2021-08-17,33,DESCARTADA,NEGATIVO,NEGATIVO,NaN,2021-08-17,2021-07-12,NaT,2021-08-18,2021-07-13,NaT,Centro de Salud Familiar Puerto Aysen,Aysen,Aysen,12719,11240,23959
21682,16074121,Coyhaique,2021-08-16,33,DESCARTADA,NaN,NEGATIVO,NaN,2021-08-16,2021-07-09,NaT,NaT,2021-07-10,NaT,Hospital Regional (Coihaique),Coyhaique,Coyhaique,28763,29055,57818
109474,16097201,Aisén,2021-08-17,33,DESCARTADA,NEGATIVO,NEGATIVO,NaN,2021-07-23,2021-08-17,NaT,2021-07-23,2021-08-18,NaT,Centro de Salud Familiar Puerto Aysen,Aysen,Aysen,12719,11240,23959
109499,16107689,Aisén,2021-08-17,33,DESCARTADA,NEGATIVO,NEGATIVO,NaN,2021-08-17,2021-07-12,NaT,2021-08-17,2021-07-13,NaT,Centro de Salud Familiar Puerto Aysen,Aysen,Aysen,12719,11240,23959
118665,16168579,Aisén,2021-08-19,33,DESCARTADA,NEGATIVO,NaN,NaN,2021-08-19,NaN,NaT,2021-08-19,NaT,NaT,Hospital de Puerto Aysén,Aysen,Aysen,12719,11240,23959


In [13]:
# Agrega 53 semanas a las semanas de 2021 y crea semana_epidemiologica_21

data_aysen_clean1 = data_aysen_clean[data_aysen_clean['fecha_notificacion']<=pd.to_datetime('2021-01-02')]
data_aysen_clean2 = data_aysen_clean[data_aysen_clean['fecha_notificacion']>=pd.to_datetime('2021-01-03')]


data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53

data_aysen_clean_21 = data_aysen_clean1.append(data_aysen_clean2, ignore_index=True)

<ipython-input-13-d2f259f8832e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
<ipython-input-13-d2f259f8832e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53


In [17]:
data_aysen_clean_21['year'] = data_aysen_clean_21['fecha_notificacion'].dt.year

data_aysen_clean_21['year']

0         2020
1         2020
2         2020
3         2020
4         2020
          ... 
125139    2021
125140    2021
125141    2021
125142    2021
125143    2021
Name: year, Length: 125144, dtype: int64

In [18]:
df_etapa_clinica = data_aysen_clean_21.set_index('semana_epidemiologica_21').groupby(['semana_epidemiologica_21','etapa_clinica','year']).count()[['numero_folio']].rename(columns={'numero_folio':'casos'}).reset_index()                   

df_etapa_clinica

,semana_epidemiologica_21,etapa_clinica,year,casos
0,10,DESCARTADA,2020,2
1,11,DESCARTADA,2020,9
2,12,CONFIRMADA,2020,1
3,12,DESCARTADA,2020,56
4,13,DESCARTADA,2020,70
...,...,...,...,...
206,87,BUSQUEDA A,2021,472
207,87,CONFIRMADA,2021,12
208,87,DESCARTADA,2021,1039
209,87,PROBABLE,2021,7


In [142]:
list(range(12,54,4))+list(range(1,88,4))

[12,
 16,
 20,
 24,
 28,
 32,
 36,
 40,
 44,
 48,
 52,
 1,
 5,
 9,
 13,
 17,
 21,
 25,
 29,
 33,
 37,
 41,
 45,
 49,
 53,
 57,
 61,
 65,
 69,
 73,
 77,
 81,
 85]

In [194]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica solo con etapa_clinica 'PROBABLE' y 'CONFIRMADA'

px.bar(df_etapa_clinica[(df_etapa_clinica['etapa_clinica']=='PROBABLE')|(df_etapa_clinica['etapa_clinica']=='CONFIRMADA')].fillna(0),
        x= 'semana_epidemiologica_21',
        y= 'casos',
        color= 'etapa_clinica',
        labels={'etapa_clinica':'Etapa Clínica'},
       ).update_layout(title= 'Casos Confirmados y Probables de la Región de Aysén',
                       yaxis_title='Número de casos',
                       xaxis_title='Semana epidemiológica',
                       template='simple_white',
                       xaxis=dict(tickmode = 'array',
                                  tickvals = list(range(12,88,3)),
                                  ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                                 ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                               ).add_vrect(x0='50', x1='50', 
                                           annotation_text='2020', 
                                           annotation_position='top right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='86', 
                                           annotation_text='2021', 
                                           annotation_position='top right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0)

In [40]:
df_etapa_clinica_2 = data_aysen_clean_21.set_index('semana_epidemiologica').groupby(['semana_epidemiologica','etapa_clinica','year']).count()[['numero_folio']].rename(columns={'numero_folio':'casos'}).reset_index()                   

df_etapa_clinica_2

,semana_epidemiologica,etapa_clinica,year,casos
0,1,CONFIRMADA,2021,185
1,1,DESCARTADA,2021,2396
2,1,PROBABLE,2021,47
3,2,BUSQUEDA A,2021,2
4,2,CONFIRMADA,2021,241
...,...,...,...,...
206,53,CONFIRMADA,2021,14
207,53,DESCARTADA,2020,1467
208,53,DESCARTADA,2021,216
209,53,PROBABLE,2020,19


In [52]:
df_etapa_clinica_2[(df_etapa_clinica_2['year']==2021)&(df_etapa_clinica_2['semana_epidemiologica']==53)]['year'] =2020

<ipython-input-52-59467f65ed45>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
df_etapa_clinica_2.loc[(df_etapa_clinica_2['year']==2021)&(df_etapa_clinica_2['semana_epidemiologica']==53)]['year']

206    2021
208    2021
210    2021
Name: year, dtype: int64

In [58]:
df_etapa_clinica_2.loc[[206,208,210],'year'] = 2020

In [59]:
df_etapa_clinica_2.loc[(df_etapa_clinica_2['year']==2021)&(df_etapa_clinica_2['semana_epidemiologica']==53)]['year']

Series([], Name: year, dtype: int64)

In [36]:
#df_etapa_clinica_2['semana_epidemiologica'] = df_etapa_clinica_2['semana_epidemiologica'].to_string()

In [110]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica solo con etapa_clinica 'PROBABLE' y 'CONFIRMADA'

px.bar(df_etapa_clinica_2[(df_etapa_clinica_2['etapa_clinica']=='PROBABLE')|(df_etapa_clinica_2['etapa_clinica']=='CONFIRMADA')],
        x= 'semana_epidemiologica',
        y= 'casos',
        color= 'etapa_clinica',
        facet_col='year',
        facet_col_spacing= 0,
     
        category_orders={'year': [2020,2021],
                         'semana_epidemiologica': list(range(1,53))
                        },
         labels={'etapa_clinica':'Etapa Clínica'},
         ).update_layout(title= 'Casos Confirmados y Probables de la Región de Aysén',
                         yaxis_title='Número de casos',
                         xaxis_title='Semana epidemiológica 2020',
                         xaxis2_title='Semana epidemiológica 2021',
                         template='simple_white',
                         yaxis2={'visible': True, 'showticklabels': False, 'ticks':''},
                         
                        ).update_xaxes(type='category', #visible=True,  
                                       ).for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))      